In [1]:
from collections import Counter
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import numpy as np
import os
import re

In [2]:
pathogens = ["Acinetobacter baumannii", "Candida albicans", "Campylobacter", "Escherichia coli", "Enterococcus faecium", "Enterobacter",
             "Helicobacter pylori", "Klebsiella pneumoniae", "Mycobacterium tuberculosis", "Neisseria gonorrhoeae", "Pseudomonas aeruginosa",
             "Plasmodium falciparum", "Staphylococcus aureus", "Schistosoma mansoni", "Streptococcus pneumoniae"]

pathogens = ["Mycobacterium tuberculosis"]
pathogens = ["Plasmodium falciparum"]
root = "."

def get_pathogen_code(pathogen):
    return str(pathogen.split()[0][0] + pathogen.split()[1]).lower() if len(pathogen.split()) > 1 else pathogen.lower()

# Get directions
directions = pd.read_csv(os.path.join(root, "..", "config", 'manual_curation', "activity_std_units_curated_manual_curation.csv"))
directions = {(i,j): k for i,j,k in zip(directions['activity_type'], directions['unit'], directions['manual_curation'])}

In [11]:
BIN = []

# For each pathogen
for pathogen in pathogens:

    # Get pathogen code
    pathogen_code = get_pathogen_code(pathogen)

    # Get assay info
    ASSAYS_INFO = pd.read_csv(os.path.join(root, "..", "output", pathogen_code, 'assays.csv'))
    ASSAYS_INFO = ASSAYS_INFO[["assay_id", "target_type", "activity_type", "unit", "activities", "nan_values", "cpds"]]
    ASSAYS_INFO = ASSAYS_INFO[ASSAYS_INFO['cpds'] >= 100]

    # Load ChEMBL bioactivity data for that pathogen
    print(f"Loading ChEMBL preprocessed data for {pathogen_code}...")
    ChEMBL = pd.read_csv(os.path.join(root, "..", "output", pathogen_code, f"{pathogen_code}_ChEMBL_data.csv"), low_memory=False)
    print(f"Number of activities for {pathogen_code}: {len(ChEMBL)}")
    print(f"Number of compounds for {pathogen_code}: {len(set(ChEMBL['compound_chembl_id']))}")

    # For each assay
    for assay_id, activity_type, unit in tqdm(zip(ASSAYS_INFO['assay_id'], ASSAYS_INFO['activity_type'], ASSAYS_INFO['unit'])):

        # Getting ChEMBL bioactivities
        if type(unit) == str:
            assay_activities = ChEMBL[(ChEMBL['assay_chembl_id'] == assay_id) & (ChEMBL['activity_type'] == activity_type) & (ChEMBL['unit'] == unit)]["value"].astype(float).tolist()
        else:
            assay_activities = ChEMBL[(ChEMBL['assay_chembl_id'] == assay_id) & (ChEMBL['activity_type'] == activity_type) & (ChEMBL['unit'].isna())]["value"].astype(float).tolist()

        # Remove nans
        assay_activities = [i for i in assay_activities if np.isnan(i) == False]
        if len(assay_activities) == 0:
            assay_activities = [np.nan]
        
        # Calculate data
        min_ = round(np.min(assay_activities), 3)
        p1 = round(np.percentile(assay_activities, 1), 3)
        p5 = round(np.percentile(assay_activities, 5), 3)
        p10 = round(np.percentile(assay_activities, 10), 3)
        p25 = round(np.percentile(assay_activities, 25), 3)
        p50 = round(np.percentile(assay_activities, 50), 3)
        p75 = round(np.percentile(assay_activities, 75), 3)
        p90 = round(np.percentile(assay_activities, 90), 3)
        p95 = round(np.percentile(assay_activities, 95), 3)
        p99 = round(np.percentile(assay_activities, 99), 3)
        max_ = round(np.max(assay_activities), 3)

        # Get direction
        direction = directions[(activity_type, unit)]

        # Store results
        BIN.append([min_, p1, p5, p10, p25, p50, p75, p90, p95, p99, max_, direction])

# To pd df
BIN = pd.DataFrame(BIN, columns=["min", "p1", "p5", "p10", "p25", "p50", "p75", "p90", "p95", "p99", "max", "direction"])
PERCENTILES = pd.concat([ASSAYS_INFO, BIN], axis=1)

# Save results
PERCENTILES.to_csv(os.path.join(root, "..", "output", pathogen_code, 'assays_activity_ranges.csv'), index=False)

Loading ChEMBL preprocessed data for pfalciparum...
Number of activities for pfalciparum: 1079517
Number of compounds for pfalciparum: 498660


100it [00:12,  7.74it/s]


In [12]:
# Count repetitions of activity_type, unit
COUNTS = PERCENTILES.groupby(["activity_type", "unit"]).size().reset_index(name="count").sort_values('count', ascending=False).reset_index(drop=True)
SUMMARY = []

# For each pair activity_type, unit
for activity_type, unit in zip(COUNTS['activity_type'], COUNTS['unit']):

    # Get direction
    direction = directions[(activity_type, unit)]

    # Get data per assay
    if type(unit) == str:
        df = PERCENTILES[(PERCENTILES['activity_type'] == activity_type) & (PERCENTILES['unit'] == unit)]
    else:
        df = PERCENTILES[(PERCENTILES['activity_type'] == activity_type) & (PERCENTILES['unit'].isna())]

    # Get only specific thresholds
    df = df[["min", "p1", "p5", "p10", "p90", "p95", "p99", "max"]]
    summary = []
    for i,j,k in zip(df.quantile(0.1, axis=0), df.quantile(0.5, axis=0), df.quantile(0.9, axis=0)):
        i,j,k = str(round(i, 3)), str(round(j, 3)), str(round(k, 3))
        summary.append(" | ".join([i,j,k]))
    
    # Append direction and store summary
    summary.append(direction)
    SUMMARY.append(summary)

# Concatenate with counts
SUMMARY = pd.DataFrame(SUMMARY, columns=["min", "p1", "p5", "p10", "p90", "p95", "p99", "max", "direction"])
COUNTS = pd.concat([COUNTS, SUMMARY], axis=1)

# Save results
COUNTS.to_csv(os.path.join(root, "..", "output", pathogen_code, 'stats_activity_ranges.csv'), index=False)

In [7]:
SMILES = ['NNC(=O)c1ccncc1' , # Isonaizid
          "CO[C@H]1/C=C/O[C@@]2(C)Oc3c(C)c(O)c4c(O)c(c(/C=N/N5CCN(C)CC5)c(O)c4c3C2=O)NC(=O)/C(C)=C\C=C\[C@H](C)[C@H](O)[C@@H](C)[C@@H](O)[C@@H](C)[C@H](OC(C)=O)[C@@H]1C",  # Rifampin
          "NC(=O)c1cnccn1",  # Pyrazinamide
          "CC[C@@H](CO)NCCN[C@@H](CC)CO"]  # Ethambutol

a, b = "INHIBITION", "%"

for smi in SMILES:

        values = ChEMBL[(ChEMBL['canonical_smiles'] == smi) & (ChEMBL['activity_type'] == a) & 
                        (ChEMBL['unit'] == b) & (ChEMBL['value'].isna() == False) & ((ChEMBL['relation'] == "=") | (ChEMBL["relation"] == '>'))]['value'].tolist()

        print(f"Number of activities: {len(values)}")
        print(np.percentile(values, 10))
        print(np.percentile(values, 50))
        print(np.percentile(values, 90))

Number of activities: 0


IndexError: index -1 is out of bounds for axis 0 with size 0